In [28]:
from openai import OpenAI
import os
from dotenv import dotenv_values
from ReadLoad import read_jsonl, write_jsonl, read_json
config = dotenv_values('.env')
client = OpenAI(
    api_key = config['qwen_key'],
    base_url = config['qwen_url']
)

def get_response(prompt):
    completion = client.chat.completions.create(
        model="qwen2-7b-instruct",
        messages=[
            {'role': 'system', 'content': "你是一个基于保险条款的问答系统，对用户提出的有关保险条款的问题给予准确、清晰的回答。"},
            {'role': 'user', 'content': prompt}
        ]
        
        )
    result = completion.choices[0].message.content
    return result


In [38]:
qa_prompt = '''从保险条款
==========
{}
==========
中找问题
==========
{}
==========
的答案，找到答案就仅使用保险条款中的语句回答问题，找不到答案就用自身知识回答并且告诉用户该信息不是来自文档。
不要复述问题，不要回答无关的内容, 直接开始回答问题。
'''

qa_prompt = '''
保险条款: """{}"""
用户问题："""{}"""
回答规则："""仅使用保险条款中的语句回答问题"""
回答: 
'''

In [45]:
data = read_json("dev.json")
for d in data:
    query = d['问题']
    clause = d['条款']
    prompt = qa_prompt.format(clause, query)
    d['prompt'] = prompt
    d['answer'] = get_response(prompt)
    break

In [46]:
data[80]

{'ID': '5080',
 '产品名': '健康福·少儿终身重疾险',
 '条款': '血液培养测试结果为阳性，证实存在感染性微生物。 本公司仅对“轻度感染性心内膜炎”和“心脏瓣膜介入手术”两项中的其中一项承担保险责任，给付其中一项保险金后，对另一项轻度疾病保险责任同时终止。11.1.11角膜移植指为增进视力或治疗某些角膜疾患，已经实施了异体的角膜移植手术。此手术必须在本公司认可的医院专科医生认为是医疗必需的情况下进行。本公司仅对“角膜移植”、“单目失明”和“视力严重受损”三项中的其中一项承担保险责任，给付其中一项保险金后，对其他两项轻度疾病保险责任同时终止。11.1.12单目失明指因疾病或意外伤害导致单眼视力永久不可逆性丧失，但未达到“双目失明”的给付标准，但患眼须满足下列至少一项条件：(1)眼球缺失或摘除；(2)矫正视力低于0.02（采用国际标准视力表，如果使用其它视力表应进行换算）；被保险人申请理赔时年龄必须在三周岁以上，且除眼球缺失或摘除情形外，应提供理赔当时的视力丧失诊断及检查证据。本公司仅对“角膜移植”、“单目失明”和“视力严重受损”三项中的11.1.13严重甲型及乙型被保险人必须罹患严重甲型血友病（缺乏VIII凝血因子）或严重血友病乙型血友病（缺乏IX凝血因子），且凝血因子VIII或凝血因子IX的活性水平少于百分之一。诊断必须由本公司认可的医院血液病专科医生确认。本公司承担本项疾病责任不受本合同责任免除中“遗传性疾病、先天性畸形、变形或染色体异常”的限制。11.1.14主动脉内手术（非为了治疗主动脉疾病实际实施了经皮经导管进行的主动脉内手开胸或开腹手术）术，且未达到“主动脉手术”的给付标准。主动脉指升主动脉、主动脉弓和降主动脉（含胸主动脉和腹主动脉），不包括升主动脉、主动脉弓和降主动脉的分支血管。11.1.15原位癌指恶性细胞局限于上皮内尚未穿破基底膜浸润周围正常组织的癌细胞新生物，且须满足以下所有条件：(1)必须经对固定活组织的组织病理学检查明确诊断，属于世界卫生组织《疾病和有关健康问题的国际统计分类》第十次修订版（ICD-10）的原位癌范畴（D00-D09）；(2)被保险人必须己经接受了针对原位癌病灶的手术治疗。癌前病变不在保障范围内。11.1.16植入心脏起搏器因严重心律失常而确实已经实施永久性心脏起搏器的植入手术。理赔时须提供完整病历资料及

In [6]:
#!pip install dashscope
import dashscope
from http import HTTPStatus
dashscope.api_key = config['qwen_key'] 

def get_embedding(embed_text):
    
    respond = ''
    try:
        respond = dashscope.TextEmbedding.call(
            model = dashscope.TextEmbedding.Models.text_embedding_v2,
            input = embed_text)
        
        embedding = respond.output['embeddings'][0]['embedding']
        
    except Exception as e:
        # Handle any other unexpected exceptions
        print(f"An unexpected error occurred: {e}")
    
    return embedding

In [18]:
!pip install tqdm
from tqdm import tqdm
for data in tqdm(dev_data):
    embed_text = data['答案']
    data['ans_embedding'] = get_embedding(embed_text)

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


100%|██████████| 1000/1000 [03:59<00:00,  4.17it/s]


In [21]:
 write_jsonl(dev_data,"dev_with_embedding")

In [17]:
text1 = "1000万航空意外险具有一年的默认保险期间"
text2 = "1000万航空意外险默认保险期间一年。"
vec1 = get_embedding(text1)
vec2 = get_embedding(text2)
similarity = cosine_similarity(vec1, vec2)
print(similarity)

0.9570271829028671


### 参考资料

https://dashscope.console.aliyun.com/billing